In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import time
import multiprocess
from multiprocess import Pool

num_procs = multiprocess.cpu_count()

In [3]:
df = pd.read_csv('data/series_icao_2019.csv')
airports = pd.read_csv('data/airports.csv')
departures = df[df["departure"].isin(airports["airport"])].copy().drop_duplicates()
arrivals = df[df["arrival"].isin(airports["airport"])].copy().drop_duplicates()
time_ = departures["dep time"].apply(lambda d: datetime.datetime.fromtimestamp(d).time() if not np.isnan(d) else "NaN")
departures["dep time"] = time_
departures["dep time minute"]  = time_.apply(lambda t: np.round(t.hour*60 + t.minute + t.second*0.1)).astype(int)

time_ = arrivals["arr time"].apply(lambda d: datetime.datetime.fromtimestamp(d).time() if not np.isnan(d) else "NaN")
arrivals["arr time"] = time_
arrivals["arr time minute"]  = time_.apply(lambda t: np.round(t.hour*60 + t.minute + t.second*0.1)).astype(int)

In [4]:
departures


,icao24,dep time,departure,arr time,arrival,callsign,dep dist,dep alt,arr dist,arr alt,candidate dep airports,candidate arr airports,day,week day,series code,series,dep time minute
0,4cc27a,14:09:40,EBBR,1554218067,BIKF,ICE67K,787.0,96.0,3061.0,36.0,2,0,2019-04-02,1,EBBR_BIKF_4cc27a_1,0,853
1,4cc27a,14:36:52,EBBR,1555428116,BIKF,ICE67K,2205.0,187.0,3053.0,206.0,2,0,2019-04-16,1,EBBR_BIKF_4cc27a_1,0,881
2,4cc27a,14:37:04,EBBR,1558452058,BIKF,ICE67K,980.0,2.0,1022.0,36.0,2,0,2019-05-21,1,EBBR_BIKF_4cc27a_1,0,877
3,4cc27c,14:16:32,EBBR,1557847278,BIKF,ICE67K,1286.0,155.0,3036.0,8.0,2,0,2019-05-14,1,EBBR_BIKF_4cc27c_1,1,859
4,4cc27c,14:35:00,EBBR,1559661550,BIKF,ICE67K,2048.0,134.0,2647.0,69.0,2,0,2019-06-04,1,EBBR_BIKF_4cc27c_1,1,875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303950,344146,15:43:50,LEMD,1563118493,LSZH,AEA1671,5943.0,53.0,1030.0,12.0,1,7,2019-07-14,6,LSZH_LEMD_344146_6,44416,948
303951,344146,08:18:30,LEMD,1563092003,LSZH,AEA76TU,5896.0,61.0,1407.0,27.0,1,7,2019-07-14,6,LSZH_LEMD_344146_6,44416,501
303952,48ada8,18:03:01,EPWA,1562521746,LSZH,LOT419,559.0,57.0,744.0,12.0,1,7,2019-07-07,6,LSZH_EPWA_48ada8_6,44417,1083
303953,48ada8,07:54:44,EPWA,1562485197,LSZH,LOT411,1396.0,133.0,589.0,12.0,1,7,2019-07-07,6,LSZH_EPWA_48ada8_6,44417,478


In [10]:
flights_departure= departures["callsign"].unique()
flights_arrival= arrivals["callsign"].unique()
flights_departure.shape

(14993,)

In [16]:
def compute(tup):
    fl_dep, dep, time_tolerance, min_occourrency = tup
    regular = 0
    for flight in fl_dep:
        f = dep[dep["callsign"]==flight]["dep time minute"]
        mean, std = f.mean(), f.std()
        if f[(f < mean + time_tolerance) & (f > mean - time_tolerance)].shape[0]/f.shape[0]> min_occourrency:
            regular += 1
    return regular

In [17]:
from itertools import product
tol = [30,35,40, 45,50,55, 60]
min_oc = [0.70,0.75,0.80,0.85,0.90,0.95]
grid = list(product(tol, min_oc))
    

## Per week day

In [18]:
day =["Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday"]
for j in range(7):
    departures_day =departures[departures["week day"]==j]
    flights_departure= departures_day["callsign"].unique()
    print(flights_departure.shape)
    len_slice = flights_departure.shape[0]//num_procs
    split_fl = [i*len_slice for i in range(num_procs)] + [flights_departure.shape[0]]
    fls = []
    t = time.time()
    for point in grid:
        partial_time = time.time()
        print(point)
        time_tolerance = point[0]
        min_occourrency = point[1]
        split_flights = tuple([(flights_departure[split_fl[i]:split_fl[i+1]], departures_day[departures_day["callsign"].isin(flights_departure[split_fl[i]:split_fl[i+1]])], time_tolerance,min_occourrency) for i in range(num_procs)])
        pool = Pool(num_procs)
        fls.append(sum(pool.map(compute, split_flights)))
        pool.close()
        pool.join()
        print("paritial", time.time()-partial_time)

    print("time total: ",time.time()-t)

    fls = np.array(fls)
    plt.rcParams["figure.figsize"] = (30,25)
    plt.rcParams["font.size"] = 25
    points = np.array(grid).T
    plt.xticks(tol)
    plt.yticks(min_oc)
    plt.xlabel("TIME TOLERANCE")
    plt.xlim(25,65)
    plt.ylim(0.65,1)
    plt.ylabel("OCCURRENCE TOLERANCE")
    plt.title(day[j])
    for i in range(len(grid)):
        plt.annotate(fls[i],(grid[i][0],grid[i][1]),color='white',horizontalalignment='center',verticalalignment='center')
    plt.scatter(points[0],points[1], s=fls*1.5)
    plt.savefig("plots/departures_icao"+day[j]+".png")
    plt.cla()
    plt.clf()
    plt.close()
    print(j)

(7810,)
(30, 0.7)
paritial 3.321540594100952
(30, 0.75)
paritial 3.396379232406616
(30, 0.8)
paritial 3.2823686599731445
(30, 0.85)
paritial 3.4370996952056885
(30, 0.9)
paritial 3.5936484336853027
(30, 0.95)
paritial 3.3571724891662598
(35, 0.7)
paritial 3.3405604362487793
(35, 0.75)
paritial 3.6906585693359375
(35, 0.8)
paritial 3.887564182281494
(35, 0.85)
paritial 4.477527856826782
(35, 0.9)
paritial 4.275025129318237
(35, 0.95)
paritial 3.5944972038269043
(40, 0.7)
paritial 3.72236967086792
(40, 0.75)
paritial 4.202314615249634
(40, 0.8)
paritial 3.638420820236206
(40, 0.85)
paritial 3.408555507659912
(40, 0.9)
paritial 3.5521748065948486
(40, 0.95)
paritial 3.7861721515655518
(45, 0.7)
paritial 3.858503818511963
(45, 0.75)
paritial 3.955970287322998
(45, 0.8)
paritial 3.7679996490478516
(45, 0.85)
paritial 3.713510036468506
(45, 0.9)
paritial 3.827530860900879
(45, 0.95)
paritial 3.789083242416382
(50, 0.7)
paritial 3.9001734256744385
(50, 0.75)
paritial 3.7589738368988037
(50, 0

paritial 2.571518898010254
(30, 0.75)
paritial 2.4536805152893066
(30, 0.8)
paritial 2.8039610385894775
(30, 0.85)
paritial 2.7468810081481934
(30, 0.9)
paritial 2.7732126712799072
(30, 0.95)
paritial 2.763990640640259
(35, 0.7)
paritial 2.6655189990997314
(35, 0.75)
paritial 2.552142858505249
(35, 0.8)
paritial 2.705169916152954
(35, 0.85)
paritial 2.5056982040405273
(35, 0.9)
paritial 2.7234702110290527
(35, 0.95)
paritial 2.8451247215270996
(40, 0.7)
paritial 3.3706259727478027
(40, 0.75)
paritial 3.2599594593048096
(40, 0.8)
paritial 3.297632932662964
(40, 0.85)
paritial 2.7109034061431885
(40, 0.9)
paritial 2.866001844406128
(40, 0.95)
paritial 3.007453441619873
(45, 0.7)
paritial 2.924006462097168
(45, 0.75)
paritial 2.8211147785186768
(45, 0.8)
paritial 2.8445544242858887
(45, 0.85)
paritial 2.6179358959198
(45, 0.9)
paritial 3.0210447311401367
(45, 0.95)
paritial 2.8119521141052246
(50, 0.7)
paritial 2.8510782718658447
(50, 0.75)
paritial 2.7628519535064697
(50, 0.8)
paritial 2

In [42]:
arrivals

,icao24,dep time,departure,arr time,arrival,callsign,dep dist,dep alt,arr dist,arr alt,candidate dep airports,candidate arr airports,day,week day,series code,series,arr time minute
0,4cc2a7,1.558355e+09,EBBR,17:11:12,BIKF,ICE67K,935.0,42.0,2909.0,39.0,2,0,2019-05-20,0,EBBR_BIKF_ICE67K _0,0,1032
1,4cc2a2,1.559569e+09,EBBR,18:20:29,BIKF,ICE67K,792.0,2.0,2624.0,69.0,2,0,2019-06-03,0,EBBR_BIKF_ICE67K _0,0,1103
2,4cc27b,1.560169e+09,EBBR,17:01:26,BIKF,ICE67K,1511.0,96.0,4266.0,69.0,2,0,2019-06-10,0,EBBR_BIKF_ICE67K _0,0,1024
3,4cc277,1.560775e+09,EBBR,17:17:20,BIKF,ICE67K,918.0,40.0,2650.0,92.0,2,0,2019-06-17,0,EBBR_BIKF_ICE67K _0,0,1039
4,3412d3,1.561983e+09,EBBR,17:30:15,BIKF,ICE67K,487.0,33.0,3049.0,6.0,2,0,2019-07-01,0,EBBR_BIKF_ICE67K _0,0,1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248337,506c21,1.567965e+09,EDDM,20:21:58,LSZH,ADR33D,2210.0,171.0,385.0,50.0,0,7,2019-09-08,6,LSZH_EDDM_ADR33D _6,73151,1227
1248338,506c21,1.568569e+09,EDDM,20:19:59,LSZH,ADR33D,1821.0,49.0,737.0,88.0,0,7,2019-09-15,6,LSZH_EDDM_ADR33D _6,73151,1225
1248339,71c251,1.567952e+09,LOWW,17:17:27,LSZH,KAL9575,1525.0,38.0,1783.0,42.0,0,7,2019-09-08,6,LSZH_LOWW_KAL9575 _6,73152,1040
1248340,71c251,1.568559e+09,LOWW,17:49:26,LSZH,KAL9575,777.0,83.0,1875.0,88.0,0,7,2019-09-15,6,LSZH_LOWW_KAL9575 _6,73152,1072


## Global std

In [47]:
def compute_std(tup):
    fl_dep, dep, tolerance = tup
    std_list = []
    
    for flight in fl_dep:
        f = dep[dep["callsign"]==flight]["dep time minute"]
        mean, std = f.mean(), f.std()
        std_list.append(std)

    return std_list

In [50]:


t = time.time()

pool = Pool(num_procs)
reg = pool.map(compute_std, split_fl)
pool.close()
pool.join()
print(time.time()-t)


9.776390552520752


In [3]:
import csv

In [6]:
import csv
file = []
with open('data/airports.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t')
    for row in spamreader:
        file.append(row)